<a href="https://colab.research.google.com/github/Brycexxx/fastai/blob/master/lesson5_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Lesson5

### 1. fine-tuning & freezing layers

<img src="https://raw.githubusercontent.com/Brycexxx/Images/master/20190216163622.jpg"/>

* 当我们在做迁移学习的时候，首先要去掉与训练模型的最后一层权重矩阵，这一层是针对特定问题的， 比如 `imagenet` 有一千个类别，所以最后一层权重矩阵的某个维度大小为 1000，但这并不是我们想要的，我们可能做一个猫狗识别的二分类，那么首先要删去最后这层权重，如上图，再加入两层随机初始化的权重矩阵用以我们的特定问题。这个时候训练需要冻结前面所有的层，因为前面权重矩阵对于图片的基础特征依然过滤得很好，我们不需要做大调整，但是最后的那一层能够组合成相当复杂的且具体的图像，这不是我们想要的，所以我们仅仅对新加入的两层权重矩阵进行更新，这时候我们的计算量下降了训练的更加快速，同时也更加节省内存

### 2. Unfreezing and Using Discriminative Learning Rates

<img src="https://raw.githubusercontent.com/Brycexxx/Images/master/20190216165016.jpg"/>

* 在冻结微调之后，进行解冻使用差异化的学习率再次训练模型。如果所示，对于非常靠前的权重矩阵，我们依然不希望有太大的改变，那些基础的特征在任何图片中都是存在的，所以我们给这些层一个非常小的学习率；然后稍后面的层，我们给一个大一点的学习率以改变针对预训练模型的一些次高级特征，调整到针对我们的目标图片
* 在 `fastai` 中，`fit_one_cycle` 方法中，`max_lr` 参数如果传入的是一个数字，那么所有层的学习率都将是 这个数字；如果传入的是 `slice(lr)` ，那么新加入的随机初始化的层的学习率将是 `lr` ，其他层都将是 `lr / 3` ；如果传入的是类似于 `slice(1e-5, 1e-3)` 那么最靠前的那些层的学习率将是 $1e-5$，最后加入的层将是 $1e-3$，中间的层将等间隔分配
*   实际上以上并不是给每层一个不同的学习率，而是将层分为几个大组，`cnn` 默认为 3 个组，默认情况下只能更新最后一个由新加入的随机初始化的权重矩阵组成的组

### 3. Affine Function（仿射函数）

* **仿射变换**，又称**仿射映射**，是指在[几何](https://zh.wikipedia.org/wiki/%E5%87%A0%E4%BD%95)中，一个[向量空间](https://zh.wikipedia.org/wiki/%E5%90%91%E9%87%8F%E7%A9%BA%E9%97%B4)进行一次[线性变换](https://zh.wikipedia.org/wiki/%E7%BA%BF%E6%80%A7%E5%8F%98%E6%8D%A2)并接上一个[平移](https://zh.wikipedia.org/wiki/%E5%B9%B3%E7%A7%BB)，变换为另一个向量空间。一个对向量 $\vec{x}$ 平移 $\vec{b}$，与旋转放大缩小 $\vec{A}$ 的仿射映射为：$\vec{y}=A\vec{x}+\vec{b}​$

* 在深度学习中，矩阵相乘是一种最常见的仿射变换

### 4. Embedding

* 原本对类别型的数据转换成数值型数据常用的是 `one-hot-encode` ，但是如果类别较多，那么得到的经过 `one-hot-encode` 变换的矩阵维度将会特别高，并且非常稀疏，造成计算时间的增加以及存储的浪费。于是可以寻找一个更加低维的表示，根据我们希望得到的低维表示随机初始化参数矩阵，然后通过反向传播可以不断更新我们的参数矩阵，最后得到最优的参数，通过和参数矩阵相乘我们就可以得到一个相对于 `one-hot-encode` 更加低维的表示。
* 将输出的 `one-hot-encode` 形式与矩阵相乘得到低维表达，这个过程耗时耗内存，通过观察可以发现，实际上原始数据正好对应着低维表达在参数矩阵中的索引，比如，原始数据为 4，那么 4 对应的低维表达向量就是参数矩阵的第四行，这意味着我们不需要做矩阵乘法，而只要做一个数组查询即可

### 5. 关于选取 `lr` 的两个技巧

* 找到 loss 曲线的最低点对应的 `lr`，然后除以 10；另一个技巧是找到斜率最大的一点。以上两种效果如果都不好，则试着扩大 10 倍或者缩小 10 倍

### 6. 训练过程中出现 `loss` 为负的情况

* 这肯定是不对的 :disappointed_relieved: 

### 7. Weight Decay

* 一种正则化手段

* 实际上就是正则化项乘的参数，也就是更常见的 $\lambda$ 

* 为什么需要正则化？

  * 正常情况下我们希望模型的复杂度正好合适，但这通常没那么容易做到，通常我们可以用参数的个数来代表模型的复杂度。当模型如下图第三个曲线，则出现了过拟合，我们为了避免过拟合，可以使用更少的参数。但是靠我们手动尝试选择出合适的参数个数以确定模型的复杂度通常比较费时，有没有什么好的办法呢？假如我们现在有 1000 个参数，模型出现了过拟合，但是如果我们把 997 个参数全部置为 $1e-9$ ，或者置为 0，那么他们对结果的贡献将会非常小了，此时我们也就能得到较好的泛化。所以，通过模型参数的数量衡量复杂度是不太对的。
  * 所以正确的衡量方式是什么？我们关心的是什么？我们为什么总是想用更多的参数？更多的参数意味着更多的非线性，意味着模型更多的可选择性。现实生活总是充满了更多曲折，但是我们不希望它的曲折和非线性超过了我们的所需。所以正确的方式是使用很多参数然后惩罚它的复杂度。如何惩罚，上面说到可以将参数置为趋于 0 的数，所以我们将所有的参数加起来让模型在训练过程中自动将某些参数置为 0。但是这貌似有一个问题，如果我们的参数有正有负，那么导致在加和的过程中会相互抵消，无法实现我们希望通过控制参数总和来控制复杂度的想法。
  * 如果我们把参数的平方加和，是不是就解决了正负抵消的问题。但是如果模型中一个非常有意义的参数非常大，那么会造成最佳损失也非常大，那么模型会趋向于将所有参数置为 0，这也不是我们想看到的。所以这个时候我们可以人工选择一个数字乘以参数的平方和以控制惩罚力度，避免所有参数为 0，这个参数在 `fastai` 中就是 `wd`。 

  <img src="https://raw.githubusercontent.com/Brycexxx/Images/master/20190217121019.png"/>

* `wd` 的默认值在 `fastai` 中取为 0.01，人们经常说默认为 0.1 较好，可实际情况是在经过多次实验发现 0.1 并不是最好的，于是 `fastai` 设为 0.01，因为在大多数情况下，你并不需要太多的参数惩罚，过大的 `wd` 会导致无论训练多久都不能很好的拟合，这个时候也就出现了欠拟合。小一点的 `wd` 你仍然可以训练的较好，可能只是刚出现过拟合而已，只要早停就好了

* 为什么叫 `weight decay` ？以上加上 $wd * \sum_1^nw_i^2$，这个形式叫做 `L2` 正则。针对这一部分对 `w` 求导则有  $2*wd*\sum_1^nW_i$，忽略前面的常数项 2，可看作原本的参数乘以一个小于 1 的数，在梯度更新的时候减去这个量，即可看作权重衰减。

### 8. 优化算法

* SGD：

  * 朴素 `sgd` 最为简单，没有动量的概念，即 $\theta_{t+1}=\theta_t-\alpha*g_t$ 
  * 缺点是收敛速度慢，可能在沟壑处震荡；合理选择学习率 $\alpha$ 是一大难点

* SGD-M：

  * `sgd` 在遇到沟壑时容易陷入震荡，此时可以引入动量 `Momentum`，可以加速 `sgd` 在正确方向的下降并抑制震荡，表达式如下：
    $$
    m_t=\beta*m_{t-1}+(1-\beta)*g_t\\
    \theta_{t+1}=\theta_t-\alpha*m_t
    $$

  * `sgd-m` 在原步长之上，增加了与上一时刻步长相关的 $\beta*m_{t-1}$，$\beta$ 通常取为 0.9 左右。这意味着参数更新方向不仅由当前的梯度决定，也与此前累积的下降方向有关。这使得参数中那些梯度方向变化不大的维度可以加速更新，并减少梯度方向变化较大的维度上的更新幅度。由此产生了加速收敛和减小震荡的效果。**具体是怎样的加速和抑制呢？** 当梯度和输入值一样分解到若干维度上时，单个维度上方向的变化只有增大或减小，此时只考虑单个维度，再看表达式，假设当前时刻和上一时刻方向一致假设都大于 0，但是当前梯度较小，但是加上 0.9 倍此前积累的正向梯度就使得最终的梯度较大，即加速了正确方向上的更新；但是如果当前梯度为负与此前积累的梯度方向相反，二者相加则产生一定的抵消，这个抵消也就抑制了震荡

  * 一个更形象的描述：一个人在路上奔跑着，这个时候你让他突然反向跑，那么实际上他是不可能立即完全反向的，只会以一条弧线偏离原来的方向，这就是动量的作用。突然反向就类似于震荡，但是在一定的速度下，是无法立刻反向的，也就在一定程度上抑制了震荡

  * 实际上 $m_t$ 的计算是一种指数加权平均。当 $\beta=0.9, t=11$ 的时候，$m_{11}$ 展开如下：

  * $$
    \begin{align}
    &m_{11}=0.1*g_{11}+0.9*m_{10}\\
    &m_{11}=0.1*g_{11}+0.1*0.9*g_{10}+0.9^2*m_{9}\\
    &m_{11}=0.1*g_{11}+0.1*0.9*g_{10}+0.1*0.9^2*g_{9}+0.9^3*m_8\\
    &.................\\
    &m_{11}=0.1*0.9^0*g_{11}+0.1*0.9^1*g_{10}+0.1*0.9^2*g_{9}......+0.1*0.9^{10}*g_1
    \end{align}
    $$

  * 观察上述推导，$0.9^{10}\approx0.35\approx\frac{1}{e}$，$g_1$ 实际上约等于 $g_{11}$ 的 1/3，再往后 $g_0$ 就不足 $g_{11}$ 的 1/3，所以对 $m_{11}$ 的贡献也就可以忽略不计。又有公式 $(1-\epsilon)^\frac{1}{\epsilon}\approx\frac{1}{e}$ ，此时$\beta=1-\epsilon$ ，所以我们可以用 $\frac{1}{1-\beta}​$ 来估计指数加权平均大概关注了往前多少个数据

  * 参考：https://www.jianshu.com/p/678d51ece537；
  [网易云课堂](https://mooc.study.163.com/learn/2001281003?tid=2001391036&_trace_c_p_k2_=5051aaa9427240909d26a5b78e23021b#/learn/content?type=detail&id=2001701051&cid=2001699117)

* SGD with Nesterov Acceleration：

  * 针对 `Momentum` 做出一点改变，并不直接使用当前的梯度 $g_t​$，而是使用即将要走的一个量 $\alpha*\beta*m_{t-1}​$ 先更新梯度即 $\theta_{i-1}-\alpha*\beta*m_{t-1}​$，到一个新的位置，然后再根据这里的梯度前进下一步，所以表示如下：

  * $$
    m_t=\beta*m_{t-1}+(1-\beta)*g(\theta_t-\alpha*\beta*m_{t-1})\\
    \theta_{t+1}=\theta_t-\alpha*m_t
    $$

  * 将以上公式变换：
    $$
    m_t=\beta*m_{t-1}+(1-\beta)*g_t+(1-\beta)*(g_{t}-g_{t-1})\\
    \theta_{t+1}=\theta_t-\alpha*m_t\\
    g_t=g(\theta_t)
    $$

  * 这个 `NAG` 的等效形式与 `Momentum` 的区别在于，本次更新方向多加了一个，它的直观含义就很明显了：如果这次的梯度比上次的梯度变大了，那么有理由相信它会继续变大下去，那我就把预计要增大的部分提前加进来；如果相比上次变小了，也是类似的情况。这样的解释听起来好像和原本的解释一样玄，但是这个多加上去的项不就是在近似目标函数的二阶导嘛！**所以NAG本质上是多考虑了目标函数的二阶导信息，怪不得可以加速收敛了！其实所谓“往前看”的说法，在牛顿法这样的二阶方法中也是经常提到的，比喻起来是说“往前看”，数学本质上则是利用了目标函数的二阶导信息**

  * 参考：https://zhuanlan.zhihu.com/p/22810533

* Adagrad：

  * 主要思想：神经网络中有大量的参数，对于经常更新的参数，我们已经积累了大量关于它们的知识，不希望它们被单个样本影响太大，希望学习速率慢一些；而对于不经常更新的参数，我们对于它们了解的信息太少，希望能从每个偶然出现的样本身上多学一些，学习速率大一些

  * 具体做法：采用二阶动量。

  * $$
    s_t=\sum_0^tg_t^2\\
    \theta_{t+1}=\theta_t-\frac{\alpha}{\sqrt{s_t+\epsilon}}*g_t(\epsilon=10^{-8} 防止分母为 0)
    $$

  * 观察上式可以发现，$\frac{1}{\sqrt{s_t}+\epsilon}$ 可以看作 $\alpha$ 的缩放比例，而经常更新的参数其二阶动量 $s_t$ 将会比较大，也就意味着 $\alpha$ 将会除以一个较大的数，学习率将被缩小，也就避免了被单个样本影响；反之，不经常更新的参数，学习率将会被放大，这样将会从单个样本上学到更多的信息

  * 存在的问题：因为二阶动量是单调递增的，所以学习率会很快减至 0 ，这样可能会使训练过程提前结束。

  * 参考：http://sakigami-yang.me/2017/12/23/GD-Series/

* Adadelta:

  * `adadelta` 的提出是为了解决 `adagrad` 学习率下降过快的问题

  * 主要思想：不使用过去所有梯度的平方和，而是使用过去固定的几个，类似于 `sgd-m`，所以表达式如下：

  * $$
    s_t=\beta*s_{t-1}+(1-\beta)*g_t^2\\
    \theta_{t+1}=\theta_t-\frac{\alpha}{\sqrt{s_t+\epsilon}}*g_t
    $$

  * 改进：作者们觉得更新的部分单位与待更新的参数单位不匹配，于是他们又定义了另一个指数平均移动更新：$u_t=\gamma*u_{t-1}+(1-\gamma)*\Delta\theta_t^2$ 

  * 但是对于当前时间 t，$\Delta\theta_t$ 还是未知的，所以使用 $\Delta\theta_{t-1}$ 代替，最终表达式如下：

  * $$
    \theta_{t+1}=\theta_t-\frac{\sqrt{u_t+\epsilon}}{\sqrt{s_t+\epsilon}}*g_t
    $$

  * 观察上式可以发现我们不再需要设置一个默认的初始学习率，公式中已经不存在了

* RMSProp:

  * 也是为了解决 `adagrad` 急剧下降的学习率

  * 实际上和 `adadelta` 的第一个版本一致：

  * $$
    s_t=\beta*s_{t-1}+(1-\beta)*g_t^2\\
    \theta_{t+1}=\theta_t-\frac{\alpha}{\sqrt{s_t+\epsilon}}*g_t
    $$

* Adam:

  * 结合 `Momentun` 和 `RMSProp`，表达如下：

  * $$
    m_t=\beta_1*m_{t-1}+(1-\beta_1)*g_t\\
    s_t=\beta_2*s_{t-1}+(1-\beta_2)*g_t^2
    $$

  * 而由于 $m_t , s_t$ 被初始化为 0，当 $\beta_1, \beta_2​$ 非常接近于 1 时，前几个计算误差非常大，所以做如下偏差修正：

  * $$
    \hat{m}_t=\frac{m_t}{1-\beta_1^t}\\
    \hat{v}_t=\frac{v_t}{1-\beta_2^t}
    $$

  * 所以最后更新公式如下：

  * $$
    \theta_{t+1}=\theta_t-\frac{\alpha}{\sqrt{\hat{v}_t}+\epsilon}*\hat{m}_t
    $$

  * 默认值分别如下：

  * $$
    \beta_1=0.9\\
    \beta_2=0.999\\
    \epsilon=10^{-8}
    $$

* 未完待续。。。。（偷懒，先歇会，不够用的时候再看大佬博客吧:joy:）详见末尾参考

* 陷入局部最优问题

  * <img src="https://raw.githubusercontent.com/Brycexxx/Images/master/20190218112246.jpg"/>
  * 如上图，当人们联想损失函数时，经常如上图左所示，z 轴表示损失，x、y 表示参数的两个维度；正如图所画，在这个损失函数上寻找最小值是非常容易陷入局部最优出不来的，但是实际上高维空间的损失函数表面并不如我们所想的三维这样。假如特征空间有 20000 个维度这么高，每个维度上要么是凹函数要么是凸函数，如果要是局部最小值，那么要求 20000 个维度都是凹的，这个概率是非常小的，所以在高维空间出现的梯度为 0 的点大都是上图右所示的鞍点
  * 所以问题不是陷入局部最优值，而是当搜索到鞍点时，梯度接近于 0 导致学习太慢，而以上多种优化算法都能够很好地解决这个问题，加速走出鞍部
  * 参考：[网易云课堂](https://mooc.study.163.com/learn/2001281003?tid=2001391036&_trace_c_p_k2_=5051aaa9427240909d26a5b78e23021b#/learn/content?type=detail&id=2001702126&cid=2001693087)
* 参考：
  * http://ruder.io/optimizing-gradient-descent/index.html
  * https://zhuanlan.zhihu.com/p/32626442

